In [1]:
import re
import numpy as np
import pandas as pd
import pymssql
import openai
from langchain_groq import ChatGroq
from MilvusRetriever import MilvusRetriever
from dotenv import load_dotenv
import yaml
from datetime import datetime, date
from functools import lru_cache
import random
import uuid

import os
import json
import dateutil
import getpass
from typing import Annotated, Optional
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

from langgraph.graph.message import AnyMessage, add_messages
from langgraph.graph import END, StateGraph, START
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_core.messages import ToolMessage, HumanMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

from consts import (
    ASSISTANT,
    COLLECT_INFO,
    VERIFY_INFORMATION,
    CLEAR_PROVIDED_INFORMATION 
)


In [2]:
load_dotenv()

True

In [3]:
retriever = MilvusRetriever(documents=[],k=3)
retriever.init()    

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/home/fermikira/anaconda3/envs/llms_env/lib/python3.12/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict =

Loading existing collection: hybrid_rag


In [4]:
# Load environment variables
load_dotenv()
data_cache = {}

# Chat history cache
chat_history_cache = {}
with open("config.yaml", 'r') as file:
        config = yaml.safe_load(file)
        database_config = config.get('database', {})
        user = database_config.get('username')
        password = database_config.get('password')


In [5]:
@lru_cache(maxsize=1000)
def load_data(Telefono):
        # Create connection
    cnxn = pymssql.connect(server='192.168.50.38\\DW_FZ', database='DW_FZ', user=user, password=password)
    # Load database configuration

    print(Telefono)
    query4 = f"""
        SELECT * FROM [DW_FZ].[dbo].[CRM_Datos_Cliente] Where Telefono = '{Telefono}';
        """
    INFO_CL = pd.read_sql_query(query4, cnxn)
    print(f"++{datetime.now()}++")
    Cedula = INFO_CL["Cedula"][0]
    
    query1 = f"""
    SELECT * FROM [DW_FZ].[dbo].[CRM_Datos_Consulta_Base] Where Cedula = {Cedula};
    """

    # Read data from database
    BASE = pd.read_sql_query(query1, cnxn)
    print(f"++{datetime.now()}++")
    Credito = BASE["Credito"].iloc[-1]
    Credito = int(Credito)

    query2 = f"""
    SELECT * FROM [DW_FZ].[dbo].[CRM_Datos_Credito] Where Credito = {Credito};
    """
    query3 = f"""
    SELECT * FROM [DW_FZ].[dbo].[CRM_Datos_Financieros] Where Credito = {Credito} ORDER BY Fecha_pago DESC;
    """
    
    CREDITOS = pd.read_sql_query(query2, cnxn)
    print(f"++{datetime.now()}++")
    PAGOS = pd.read_sql_query(query3, cnxn)
    print(f"++{datetime.now()}++")

    return BASE, CREDITOS, PAGOS, INFO_CL

In [6]:
#BASE, CREDITOS, PAGOS, INFO_CL = load_data(3152332041)
#INFO_CL

In [7]:
@tool
def validar_cedula(cedula, Telefono):
    """Valida la cedula de la eprsona con el telefono

    Args:
        cedula (_type_): _description_
        Telefono (_type_): _description_

    Returns:
        _type_: _description_
    """
    try:
        BASE, _, _, _ = load_data(Telefono)
        cedulas_validas = BASE['Cedula'].astype(str)
        
        print(f"Cédula a validar: {cedula}")
        print(f"Cédulas válidas en la base de datos: {cedulas_validas}")
        
        if str(cedula) in str(cedulas_validas):
            print(f"Cédula {cedula} validada correctamente.")
            return True
        else:
            print(f"Cédula {cedula} no encontrada en la base de datos.")
            return False
    except Exception as e:
        print(f"Error al validar cédula: {e}")
        return False

In [8]:
@tool
def validar_telefono(Telefono):
    """Valida el telefono de la persona
    """
    try:
        cnxn = pymssql.connect(server='192.168.50.38\\DW_FZ', database='DW_FZ', user=user, password=password)
        query4 = f"""
        SELECT * FROM [DW_FZ].[dbo].[CRM_Datos_Cliente] Where Telefono = '{Telefono}';
        """
        INFO_CL = pd.read_sql_query(query4, cnxn)
        
        print(f"Cédula a validar: {cedula}")
        print(f"Cédulas válidas en la base de datos: {cedulas_validas}")
        
        if not INFO_CL.empty():
            print(f"Tiene cuenta validada correctamente.")
            return True
        else:
            print(f"Cuenta no encontrada en la base de datos.")
            return False
    except Exception as e:
        print(f"Error al validar cédula: {e}")
        return False

In [9]:
@tool  
def obtener_creditos(cedula):
    """ Esta función obtiene los créditos"""
    try:
        cnxn = pymssql.connect(server='192.168.50.38\\DW_FZ', database='DW_FZ', user=user, password=password)
        query = f"""
        SELECT Credito, Cedula, Nombre, rol, Placa, Estado_credito 
        FROM [DW_FZ].[dbo].[CRM_Datos_Consulta_Base] 
        WHERE Cedula = '{cedula}'
        """
        creditos = pd.read_sql_query(query, cnxn)
        return creditos
    except Exception as e:
        print(f"Error al obtener créditos: {e}")
        return pd.DataFrame()

# Working tools

In [10]:
@tool
def lookup_questions(query : str) -> str:
    """
    Consulta la base de datos de documentos para resolver la pregunta del cliente
    """
    docs = retriever.invoke(query)
    return docs

In [11]:

@tool
def mostrar_creditos(creditos):
    """
    Muestra los creditos que tiene el cliente
    """
    if creditos.empty:
        return "No se encontraron créditos asociados a esta cédula."
    
    creditos_vigentes = creditos[creditos['Estado_credito'] == 'Vigente']
    if not creditos_vigentes.empty:
        creditos_mostrar = creditos_vigentes
    else:
        creditos_mostrar = creditos
    
    mensaje = "Estos son sus créditos:\n\n"
    for _, credito in creditos_mostrar.iterrows():
        mensaje += f"Crédito: {credito['Credito']}\n"
        mensaje += f"Estado: {credito['Estado_credito']}\n"
        mensaje += f"Placa: {credito['Placa'] if credito['Placa'] != '0' else 'No aplica'}\n\n"
    
    return mensaje


In [12]:

@tool
def extraer_cedula(user_query: str) -> dict:
    """
    Extrae la cedula del mensaje del usuario siguiente
    """
    template = f"""
    Extrae el número de cédula del siguiente mensaje del cliente.
    Mensaje del cliente: {user_query}
    Responde únicamente con un objeto JSON que contenga una sola clave "cedula" y el valor de la cédula extraída.
    Si no hay un número de cédula claro, responde con "cedula": "no_encontrada".
    Ejemplo de formato:
    '''
        "cedula": "123456789"
    '''
    IMPORTANTE:
    - Responde SOLO con el objeto JSON.
    - No incluyas explicaciones adicionales.
    - Asegúrate de que tu respuesta sea válida en formato JSON.
    """
    prompt = ChatPromptTemplate.from_template(template)
    llm = ChatGroq(groq_api_key=os.environ['GROQ_API_KEY_2'], model_name="llama3-70b-8192")
    chain = prompt | llm | JsonOutputParser()
    response = chain.invoke({"user_query": user_query})
    
    try:
        return response
    except Exception as e:
        print(f"Error parsing JSON response: {e}")
        return {"cedula": "no_encontrada"}

In [13]:

@tool
def safe_convert(data):
    """
    Convierte datos en json
    Args:
        data (_type_): _description_

    Returns:
        _type_: _description_
    """
    try:
        return json.dumps(data, default=str)
    except Exception as e:
        print(f"Error converting data: {data} - {e}")
        return str(data)  # Fallback en caso de error

In [14]:


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

# GRAFO

In [15]:
class RequiredInformation(BaseModel):
    provided_id: Optional[int] = Field(None,description="La cédula que proporcionó el usuario")
    provided_email: Optional[str] = Field(None,description="El email que proporcionó el usuario")

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    validated: bool
    required_information: RequiredInformation
    telefono : int

llm = ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'], model_name="llama3-70b-8192")

In [16]:
class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State):
        print("En el asistente!!!")
        while True:
            result = self.runnable.invoke(
                    {
                        "user_question":state["messages"][-1],
                        "messages": state["messages"] if "messages" in state else [],
                        "telefono": state["telefono"]
                    }
                )
            # If the LLM happens to return an empty response, we will re-prompt it
            # for an actual response.
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Responde con un output real")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


system = """Eres un asistente auxiliar con la tarea de verificar la identidad del cliente.
            1. Primero necesitas recoger la información del cliente para poder verificarlo.
            2. Luego de que colectes toda la información, di amablemente gracias y que vas a pasar a verificarlo.
            
            La información a continuación es la que debes recolectar:

            class RequiredInformation(BaseModel):
                provided_id: Optional[int] = Field(description="La cédula que proporcionó el usuario")
                provided_email: Optional[str] = Field(description="El email que proporcionó el usuario")
                
            Asegurate de tener la información antes de que puedas proceder, pero recolectala un campo a la vez. 
            Si el usuario se equivocó ingresando los datos, por favor dile porqué y que vuelva a ingresar el dato.
            Si alguna de esta información no es proporcionada retorna None

            NO LLENES LA INFORMACIÓN DEL USUARIO, RECOLECTALA.
            """
assistant_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "User question: {user_question}"
            "Chat history: {messages}"
            "Telefono:{telefono}"
            "\n\n What the user have provided so far {provided_required_information} \n\n",
        ),
    ]
)

def assistant_node(state: State) -> dict:
    print("En el logueador")
    get_information_chain = assistant_prompt | llm
    res = get_information_chain.invoke(
        {
            "user_question":state["messages"][-1],
            "provided_required_information": state["required_information"] if "required_information" in state else None,
            "messages": state["messages"] if "messages" in state else [],
            "telefono": state["telefono"]
        }
    )

    return {"messages": [res]}

def combine_required_info(info_list: list[RequiredInformation]) -> RequiredInformation:
    print("Combinando información requerida...")
    info_list = [info for info in info_list if info is not None]

    if len(info_list) == 1:
        return info_list[0]
    combined_info = {}
    for info in info_list:
        for key, value in info.model_dump().items():
            if value is not None:
                combined_info[key] = value
    print(combined_info)
    return RequiredInformation(**combined_info)

def collect_info(state: State) -> dict:
    print("Recolectando información...")
    information_from_stdin = str(input("\nenter information\n"))
    structured_llm_user_info = llm.with_structured_output(RequiredInformation)

    information_chain = assistant_prompt | structured_llm_user_info
    res = information_chain.invoke(
        {
            "user_question": state["messages"][-1],
            "provided_required_information": information_from_stdin,
            "messages": state["messages"],
            "telefono": state["telefono"]
        }
    )
    if "required_information" in state:
        required_info = combine_required_info(
            info_list=[res, state.get("required_information")]
        )
    else:
        required_info = res
    return {
        "required_information": required_info,
        "messages": [HumanMessage(content=information_from_stdin)],
    }
    
def verify_information(state: State) -> dict:
    print("Verificando...")
    Telefono = state["telefono"]
    required_information: RequiredInformation = state["required_information"]
    print(required_information)
    cnxn = pymssql.connect(server='192.168.50.38\\DW_FZ', database='DW_FZ', user=user, password=password)
    query4 = f"""
        SELECT * FROM [DW_FZ].[dbo].[CRM_Datos_Cliente] Where Telefono = '{Telefono}';
        """
    df_cl = pd.read_sql_query(query4, cnxn)
    if not df_cl.empty:
        correo_cl = df_cl['Correo']
        cedula_cl = df_cl['Cedula']
        if required_information.provided_id == cedula_cl.values[0] and required_information.provided_email == correo_cl.values[0]:
            print("Verificado!!!")
            return {"validated": True}
        else:
            return {"validated": False}          
    else: 
        return {"validated": False}
    
def provided_all_details(state: State) -> str:
    print("Mirando si ya ingresó toda la info")
    if "required_information" not in state:
        return "need to collect more information"
    provided_information: RequiredInformation = state["required_information"]
    if (
        provided_information.provided_id
        and provided_information.provided_email
    ):
        print("Ya ingresó toda la info")
        return "all information collected"

    else:
        return "need to collect more information"


def verified(state: State) -> str:
    print("En la arista de verificación")
    verified_successfully = state["validated"]

    if verified_successfully:
        return "agent_with_tools"
    else:
        return ASSISTANT
        
    

In [17]:
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres SAC, el agente de servicio al cliente más eficiente de Finanzauto en Colombia."
            "Usa las herramintas otorgadas para responder a las preguntas del usuario, mostrar creditos, etc."
            "\n\nteléfono del usuario actual:\n<User>\n{telefono}\n</User>"
            "\nTiempo actual: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now())

part_1_tools = [
    lookup_questions
]
part_1_assistant_runnable = primary_assistant_prompt | llm.bind_tools(part_1_tools)

In [18]:
workflow = StateGraph(State)
workflow.add_node(ASSISTANT, assistant_node)
workflow.add_node(COLLECT_INFO, collect_info)
workflow.add_node(VERIFY_INFORMATION, verify_information)
workflow.add_node("agent_with_tools", Assistant(part_1_assistant_runnable))


workflow.set_entry_point(ASSISTANT)
workflow.add_edge(ASSISTANT, COLLECT_INFO)
workflow.add_conditional_edges(
    COLLECT_INFO,
    provided_all_details,
    {
        "need to collect more information": "assistant",
        "all information collected": "verify_information",
    },
)
workflow.add_conditional_edges(
    VERIFY_INFORMATION,
    verified,
    {"agent_with_tools": "agent_with_tools", ASSISTANT: ASSISTANT},
)
workflow.add_edge("agent_with_tools", END)
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory, interrupt_after=['collect_info',])
graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

In [19]:
graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x01\xd8ICC_PROFILE\x00\x01\x01\x00\x00\x01\xc8\x00\x00\x00\x00\x040\x00\x00mntrRGB XYZ \x07\xe0\x00\x01\x00\x01\x00\x00\x00\x00\x00\x00acsp\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tdesc\x00\x00\x00\xf0\x00\x00\x00$rXYZ\x00\x00\x01\x14\x00\x00\x00\x14gXYZ\x00\x00\x01(\x00\x00\x00\x14bXYZ\x00\x00\x01<\x00\x00\x00\x14wtpt\x00\x00\x01P\x00\x00\x00\x14rTRC\x00\x00\x01d\x00\x00\x00(gTRC\x00\x00\x01d\x00\x00\x00(bTRC\x00\x00\x01d\x00\x00\x00(cprt\x00\x00\x01\x8c\x00\x00\x00<mluc\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x0cenUS\x00\x00\x00\x08\x00\x00\x00\x1c\x00s\x00R\x00G\x00BXYZ \x00\x00\x00\x00

In [20]:
display(Image(graph.get_graph().draw_mermaid_png()))

"%%{init: {'flowchart': {'curve': 'linear'}}}%%\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\tassistant(assistant)\n\tcollect_info(collect_info<hr/><small><em>__interrupt = after</em></small>)\n\tverify_information(verify_information)\n\tagent_with_tools(agent_with_tools)\n\t__end__([<p>__end__</p>]):::last\n\t__start__ --> assistant;\n\tagent_with_tools --> __end__;\n\tassistant --> collect_info;\n\tcollect_info -. &nbsp;need to collect more information&nbsp; .-> assistant;\n\tcollect_info -. &nbsp;all information collected&nbsp; .-> verify_information;\n\tverify_information -.-> agent_with_tools;\n\tverify_information -.-> assistant;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n"

In [21]:
thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        "thread_id": 1,
    }
}
for event in graph.stream({"messages": ["Hola! Mi nombre es Daniel"], "telefono":'3152332041'}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hola! Mi nombre es Daniel
En el logueador
================================== Ai Message ==================================

Hola Daniel! Para verificar tu identidad, necesito algunos datos más. ¿Podrías proporcionarme la cédula que tienes?
Recolectando información...
Mirando si ya ingresó toda la info
================================ Human Message =================================

125748


In [22]:
for event in graph.stream(None, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

125748
En el logueador
================================== Ai Message ==================================

¡Hola Daniel! Gracias por proporcionarme tu cédula, que es 125748. Ahora, necesito tu dirección de correo electrónico para verificar tu identidad. ¿Podrías proporcionarme tu email, por favor?
Recolectando información...
Combinando información requerida...
{'provided_id': 125748}
Mirando si ya ingresó toda la info
================================ Human Message =================================

juanfonsecagaravito@gmail.com


In [23]:
for event in graph.stream(None, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

juanfonsecagaravito@gmail.com
En el logueador
================================== Ai Message ==================================

¡Hola Daniel! Gracias por proporcionarme tu cédula y correo electrónico. Ahora, voy a verificar la información que me has proporcionado.

He recibido la siguiente información:

* La cédula que proporcionaste es: 125748
* El email que proporcionaste es: juanfonsecagaravito@gmail.com

Voy a verificar esta información para asegurarme de que sea correcta. ¡Un momento, por favor!
Recolectando información...
Combinando información requerida...
{'provided_id': 125748, 'provided_email': 'juanfonsecagaravito@gmail.com'}
Mirando si ya ingresó toda la info
Ya ingresó toda la info
================================ Human Message =================================




In [24]:
for event in graph.stream(None, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================


Verificando...
provided_id=125748 provided_email='juanfonsecagaravito@gmail.com'


/tmp/ipykernel_417823/631530422.py:120: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cl = pd.read_sql_query(query4, cnxn)


Verificado!!!
En la arista de verificación
================================ Human Message =================================


En el asistente!!!
================================== Ai Message ==================================
Tool Calls:
  lookup_questions (call_b2vq)
 Call ID: call_b2vq
  Args:
    query: Verificar cédula y correo electrónico


In [25]:
for event in graph.stream({'messages':"donde queda la sede de medellín?"}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

donde queda la sede de medellín?
En el logueador
================================== Ai Message ==================================

¡Hola Daniel! 

Ya tengo toda la información que necesito para verificar tu identidad. Gracias por proporcionarme tu cédula y correo electrónico.

He recibido la siguiente información:

* La cédula que proporcionaste es: 125748
* El email que proporcionaste es: juanfonsecagaravito@gmail.com

Voy a verificar esta información para asegurarme de que sea correcta. ¡Un momento, por favor!
Recolectando información...
Combinando información requerida...
{'provided_id': 125748, 'provided_email': 'juanfonsecagaravito@gmail.com'}
Mirando si ya ingresó toda la info
Ya ingresó toda la info
================================ Human Message =================================




In [26]:
for event in graph.stream(None, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================


Verificando...
provided_id=125748 provided_email='juanfonsecagaravito@gmail.com'


/tmp/ipykernel_417823/631530422.py:120: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cl = pd.read_sql_query(query4, cnxn)


Verificado!!!
En la arista de verificación
================================ Human Message =================================


En el asistente!!!
================================== Ai Message ==================================
Tool Calls:
  lookup_questions (call_2yk9)
 Call ID: call_2yk9
  Args:
    query: Sede de Medellín


In [27]:
for event in graph.stream(None, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  lookup_questions (call_2yk9)
 Call ID: call_2yk9
  Args:
    query: Sede de Medellín


In [28]:
for event in graph.stream(None, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  lookup_questions (call_2yk9)
 Call ID: call_2yk9
  Args:
    query: Sede de Medellín
